In [1]:
# Copyright 2019, The TensorFlow Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Training a CNN on MNIST with Keras and the DP SGD optimizer."""

from absl import app
from absl import flags
from absl import logging
import dp_accounting
import numpy as np
import tensorflow as tf
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer


flags.DEFINE_boolean(
    "dpsgd", True, "If True, train with DP-SGD. If False, " "train with vanilla SGD."
)
flags.DEFINE_float("learning_rate", 0.15, "Learning rate for training")
flags.DEFINE_float(
    "noise_multiplier", 0.1, "Ratio of the standard deviation to the clipping norm"
)
flags.DEFINE_float("l2_norm_clip", 1.0, "Clipping norm")
flags.DEFINE_integer("batch_size", 250, "Batch size")
flags.DEFINE_integer("epochs", 30, "Number of epochs")
flags.DEFINE_integer(
    "microbatches", 250, "Number of microbatches " "(must evenly divide batch_size)"
)
flags.DEFINE_string("model_dir", None, "Model directory")

FLAGS = flags.FLAGS

In [2]:
    """Loads MNIST and preprocesses to combine training and validation data."""
    train, test = tf.keras.datasets.mnist.load_data()


In [3]:
train_data, train_labels = train

In [13]:
train_data.shape


(60000, 28, 28)

In [12]:
    train_data, train_labels = train
    test_data, test_labels = test

    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

In [14]:
    train_data = train_data.reshape((train_data.shape[0], 28, 28, 1))
    test_data = test_data.reshape((test_data.shape[0], 28, 28, 1))


In [15]:
train_data.shape

(60000, 28, 28, 1)

In [ ]:

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

    assert train_data.min() == 0.0
    assert train_data.max() == 1.0
    assert test_data.min() == 0.0
    assert test_data.max() == 1.0

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [3]:
train_images.shape

(50000, 32, 32, 3)